<a href="https://colab.research.google.com/github/smithkanzariya/Fake_News_Classifier-Using-Bidirectional-LSTM/blob/main/FakeNewsClassifierUsingBidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Loading Dataset
import pandas as pd
Dataset  = pd.read_csv("/content/drive/MyDrive/News/train.csv")
Dataset.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


Dealing with Nan Values

In [3]:
Dataset.shape

(20800, 5)

In [4]:
Dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
#Drop NAN Values
Dataset = Dataset.dropna()

In [6]:
Dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
Dataset.shape

(18285, 5)

Deviding the Dataset into Dependent and Independent

In [8]:
#Get the independent Feature
X = Dataset.drop("label", axis = 1)
X.shape
Y = Dataset["label"]
Y.shape

(18285,)

In [9]:
Y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [10]:
X.shape


(18285, 4)

In [11]:
#Importing the tensorflow library
import tensorflow as tf 

In [12]:
tf.__version__

'2.8.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  Bidirectional

In [14]:
#Vocabulary size
voc_size = 5000

Onehot Representaion

In [15]:
messages = X.copy()
messages.reset_index(inplace = True)
messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
#Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  #print(i)
  review = re.sub("[^a-zA-Z]", " ", messages["title"][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

In [19]:
corpus[:100]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[:100]

[[544, 4384, 712, 645, 3361, 824, 4460, 4611, 1604, 3127],
 [1429, 1290, 2191, 330, 3627, 2701, 798],
 [3364, 3830, 2334, 2193],
 [1991, 1054, 247, 1793, 3738, 2587],
 [3911, 3627, 4535, 3440, 4566, 4939, 3627, 4726, 660, 4409],
 [1049,
  2,
  167,
  4679,
  2320,
  3034,
  3917,
  1461,
  3341,
  4401,
  78,
  1283,
  2574,
  1223,
  798],
 [2121, 1128, 4796, 648, 1859, 2948, 3443, 1314, 3118, 4962, 1031],
 [2522, 4446, 557, 2090, 184, 1892, 3034, 2832, 3118, 4962, 1031],
 [4921, 4117, 1331, 3830, 4650, 2398, 1350, 4574, 3034, 3686],
 [2378, 2165, 1288, 2846, 199, 2353, 1215, 4182],
 [4681, 796, 1481, 1705, 2726, 2445, 580, 2864, 696, 2480, 2602],
 [1793, 2599, 3361, 2398, 3034, 184],
 [3668, 2392, 2725, 3224, 105, 185, 1390, 2827, 2429],
 [1886, 4152, 3600, 315, 779, 3751, 124, 3118, 4962, 1031],
 [3353, 2301, 2020, 2035, 513, 3118, 4962, 1031],
 [4811, 49, 1313, 2183, 2081, 4881, 1278, 4055, 4783, 3730],
 [1107, 378, 1290],
 [4846, 2802, 132, 3406, 3034, 779, 367, 798],
 [622, 3124,

In [21]:
longest = max(onehot_repr, key = len)
print(len(longest))

47


Embedding Rpresentation

In [22]:
sent_lenght = 20
embedded_docs = pad_sequences(onehot_repr, padding="pre", maxlen= sent_lenght)
print(embedded_docs)

[[   0    0    0 ... 4611 1604 3127]
 [   0    0    0 ... 3627 2701  798]
 [   0    0    0 ... 3830 2334 2193]
 ...
 [   0    0    0 ... 3118 4962 1031]
 [   0    0    0 ... 1662 1283 1820]
 [   0    0    0 ... 3618 2163 1841]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  544,
       4384,  712,  645, 3361,  824, 4460, 4611, 1604, 3127], dtype=int32)

Build The Model

In [24]:
from tensorflow.keras.layers import Dropout
#Creating model
embedding_vector_feature = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_lenght))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation= "sigmoid"))
model1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [27]:
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [28]:
#Splitng the data into train and test.
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

In [29]:
X_train.shape, Y_train.shape

((12250, 20), (12250,))

Model Training

In [30]:
#Training Time
model1.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 15s 56ms/step - loss: 0.2913 - accuracy: 0.8660 - val_loss: 0.2026 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1344 - accuracy: 0.9495 - val_loss: 0.2080 - val_accuracy: 0.9170
Epoch 3/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0973 - accuracy: 0.9642 - val_loss: 0.2281 - val_accuracy: 0.9145
Epoch 4/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0711 - accuracy: 0.9758 - val_loss: 0.2828 - val_accuracy: 0.9077
Epoch 5/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0569 - accuracy: 0.9808 - val_loss: 0.2983 - val_accuracy: 0.9114
Epoch 6/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0365 - accuracy: 0.9891 - val_loss: 0.3098 - val_accuracy: 0.9036
Epoch 7/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0355 - accuracy: 0.9873 - val_loss: 0.3749 - val_accuracy:

Performence Metrics and Accuracy 

In [31]:

Y_pred1 = (model1.predict(X_test) > 0.5).astype("int32")

In [32]:
Y_pred1

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [33]:
len(Y_pred1)

6035

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
type(Y_test), type(Y_pred1)

(numpy.ndarray, numpy.ndarray)

In [36]:
confusion_matrix(Y_test, Y_pred1)

array([[3085,  334],
       [ 245, 2371]])

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred1)

0.904059652029826

In [38]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test, Y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

